## 0. Подготовительная часть ##

In [2]:
import pandas as pd
import re
import datetime as dt
import json
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.integrate as integrate
from sklearn import linear_model

In [192]:
FILE_TOP500_DYNAMICS_LIST = 'data/top500_stat'
FILE_TOP500_CURRENT = 'data/TOP500_202206.csv'
FORECAST_DATE = dt.date(2025, 11, 1)
ENERGY_PRODUCTION = 26823200 # GWh

In [130]:
FLOP_SCALE = {'G': 1, 'T' : 1000, 'P':1000000, 'E': 1000000000}

## 1. На основании данных рейтинга top500 спрогнозировать  производительность компьютера №1 в 2025 ##

In [137]:
with open(FILE_TOP500_DYNAMICS_LIST, 'r') as ifile:
    content = ifile.read()
    table = re.findall(r'\"(\d\d)\/(\d\d\d\d).*#1\s=\s(.*)\s.*([A-Z]).*Flop.*\"', content)

predata = {dt.date(int(i[1]), int(i[0]), 1) : float(i[2]) * FLOP_SCALE[i[3]] for i in table}
min_date = min(data.keys())
data = {i[0] - min_date : i[1] for i in predata.items()}

In [154]:
X_train = np.array([x.days for x in data.keys()]).reshape(-1,1)
y_train = np.array([np.log(x) for x in data.values()])

lr_model = linear_model.LinearRegression()
lr_model.fit(X_train, y_train)
lr_model.score(X_train, y_train), lr_model.coef_[0], lr_model.intercept_

(0.9873899745487242, 0.0016012235309328887, 4.4365750956123335)

In [155]:
lr_predicted = lr_model.predict(np.array((FORECAST_DATE - min_date).days).reshape(-1,1))
np.exp(lr_predicted[0])

14489106459.82071

### Пиковый суперкомпьютер к рейтингу на ноябрь 2025 будет иметь мощность 14489106459.82071 GFlop/s = 14.5 EFlop/s

## 2. оцените какой процент всего производимого на Земле электричества потребляют все компьютеры из top500 (на основании открытых данных top500.org) ##

In [171]:
df = pd.read_csv(FILE_TOP500_CURRENT, sep=';', decimal=",")

In [172]:
df.columns

Index(['Rank', 'Previous Rank', 'First Appearance', 'First Rank', 'Name',
       'Computer', 'Site', 'Manufacturer', 'Country', 'Year', 'Segment',
       'Total Cores', 'Accelerator/Co-Processor Cores', 'Rmax [TFlop/s]',
       'Rpeak [TFlop/s]', 'Nmax', 'Nhalf', 'HPCG [TFlop/s]', 'Power (kW)',
       'Power Source', 'Energy Efficiency [GFlops/Watts]', 'Memory',
       'Architecture', 'Processor', 'Processor Technology',
       'Processor Speed (MHz)', 'Operating System', 'OS Family',
       'Accelerator/Co-Processor', 'Cores per Socket', 'Processor Generation',
       'System Model', 'System Family', 'Interconnect Family', 'Interconnect',
       'Continent', 'Site ID', 'System ID'],
      dtype='object')

In [174]:
mean_power_consuption = np.mean(df[df['Energy Efficiency [GFlops/Watts]'].notnull()]['Energy Efficiency [GFlops/Watts]'])
mean_power_consuption

8.584869109947643

In [184]:
rmaxes = [float(i.replace(' ', '').replace(',','.')) for i in df['Rmax [TFlop/s]']]

In [185]:
total_rmax = np.sum(rmaxes)
total_rmax

4403083.21

In [187]:
print(mean_power_consuption / 1000 , "TFlops/Watts")

0.008584869109947643 TFlops/Watts


In [189]:
energy_consumption = total_rmax / (mean_power_consuption / 1000)
energy_consumption

512888799.3059749

In [190]:
print(energy_consumption / 1e9, 'GWatts')

0.5128887993059749 GWatts


In [194]:
print('Доля от мирового производства энергии',(energy_consumption / 1e9) / ENERGY_PRODUCTION *100, '%')

Доля от мирового производства энергии 1.9121089180484615e-06 %


## 3. Бонус: Улучшить производительность программы с реализацией алгоритма матричного умножения ##